In [ ]:
import pandas as pd
import random
import numpy as np
from operator import itemgetter
from collections import Counter
import datetime
from scipy import interp
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly 
import missingno as msno



In [ ]:
def read_data():
    import sherlockml.filesystem as sfs
    #sfs.get('/input/spotfunc.py', 'spotfunc.py')
    sfs.get('/input/cleaned_data.csv', 'cleaned_data.csv')

    %%time
    # Read in sampled data
    data = pd.read_csv('cleaned_data.csv')
    print('rows:',len(data))
    data.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','upc'],axis=1,inplace=True)
    return data

def get_user_streams(df):
    sc = df.groupby('customer_id')['customer_id'].count().to_dict()
    df['user_stream_count'] = df.customer_id.map(sc)
    return df

def get_listening_days(df):
    dc = df.groupby('customer_id')['date'].nunique().to_dict()
    df['listening_days'] = df.customer_id.map(dc)
    return df

def get_num_songs(df):
    songs = df.groupby('customer_id')['track_id'].nunique().to_dict()
    df['num_songs'] = df.customer_id.map(songs)
    return df

def get_num_albums(df):
    album = df.groupby('customer_id')['album_name'].nunique().to_dict()
    df['num_albums'] = df.customer_id.map(album)
    return df

def get_num_playlists(df):
    p = df.groupby('customer_id')['playlist_id'].nunique().to_dict()
    df['num_playlists'] = df.customer_id.map(p)
    return df

def get_num_artists(df):
    artists = df.groupby('customer_id')['artist_name'].nunique().to_dict()
    df['num_artists'] = df.customer_id.map(artists)
    return df

def get_age_bins(data):
    data['birth_year']=data['birth_year'].dropna()
    data['age'] = 2017- data.birth_year
    data['age_bin'] = pd.cut(data['age'], [0, 7, 12, 18, 25, 40, 60, 120], labels=['baby', 'toddler', 'teens', 
                                                                              'young adult', 'adult','old','oldest'])
    data = pd.get_dummies(data,columns=['age_bin'])
    return data

def get_songs_age(data):
    first_played = data.groupby('track_id')['date'].min().to_dict()
    last_played = data.groupby('track_id')['date'].max().to_dict()
    data['song_first_played'] = data.track_id.map(first_played)
    data['song_last_played'] = data.track_id.map(last_played)
    data['song_lifetime'] = data.song_last_played - data.song_first_played
    data['song_lifetime'] = (data.song_lifetime / np.timedelta64(1, 'D')).astype(int)
    return data

def get_albums_age(data):
    first_played = data.groupby('album_name')['date'].min().to_dict()
    last_played = data.groupby('album_name')['date'].max().to_dict()
    data['album_first_played'] = data.album_name.map(first_played)
    data['album_last_played'] = data.album_name.map(last_played)
    data['album_lifetime'] = data.album_last_played - data.album_first_played
    data['album_lifetime'] = (data.album_lifetime / np.timedelta64(1, 'D')).astype(int)
    return data

def get_songs_stream_count(data):
    sc = data.groupby('track_id')['track_id'].count().to_dict()
    data['song_stream_count'] = data.track_id.map(sc)
    return data

def get_albums_stream_count(data):
    ac = data.groupby('album_name')['album_name'].count().to_dict()
    data['album_stream_count'] = data.album_name.map(ac)
    return data

def get_artist_data(artist_name):
    artist = data[data.artist_name==artist_name]
    artist['date']= pd.to_datetime(artist.date, errors='coerce').dt.date
    artist = get_user_streams(artist)
    artist = get_songs_stream_count(artist)
    artist = get_albums_stream_count(artist)
    # get first listen, last listen, and cohort group
    artist = set_cohort_group(artist)
    artist = get_listening_days(artist)
    
    artist['lifetime'] = artist.last_listen - artist.first_listen 
    artist['lifetime'] = (artist.lifetime / np.timedelta64(1, 'D')).astype(int)
    artist['lifetime_bin'] =  pd.cut(artist.lifetime, [-1, 31,364, 365, 730, 900], labels=['1 month', '>1 month',
                                                                                        '1 year','1-2 year','>2 years'])
    artist = get_num_albums(artist)
    artist = get_num_songs(artist)
    artist = get_num_playlists(artist)
    artist = get_age_bins(artist)
     
    artist = set_cohort_group(artist)
    
    artist['first_listen'] = pd.to_datetime(artist.first_listen, errors='coerce')
    artist['first_day'] = datetime.datetime.now() - artist['first_listen']
    artist['first_day'] = (artist.first_day / np.timedelta64(1, 'D')).astype(int)
    
    artist = get_songs_age(artist)
    artist = get_albums_age(artist)
    
    print(artist.head)
    return artist

def get_attributes(artist):
    # run this first before filtering based on artist name
    artist['date']= pd.to_datetime(artist.date, errors='coerce').dt.date
    print('get stream count')
    artist = get_user_streams(artist)
    artist = get_songs_stream_count(artist)
    artist = get_albums_stream_count(artist)
    # get first listen, last listen, and cohort group
      
    print('get customers listening stats')
    artist = get_num_albums(artist)
    artist = get_num_songs(artist)
    artist = get_num_playlists(artist)
    #artist = get_num_artists(artist)
    
    print('get other stats')
    artist = get_age_bins(artist)
    artist = get_listening_days(artist)  
    
    
    print('get other stats')
    artist = get_songs_age(artist)
    artist = get_albums_age(artist)
    #artist = get_playlists_age(artist)
    
    print('get cohort group')
    artist = set_cohort_group(artist)
    
    artist['first_listen'] = pd.to_datetime(artist.first_listen, errors='coerce')
    artist['first_day'] = datetime.datetime.now() - artist['first_listen']
    artist['first_day'] = (artist.first_day / np.timedelta64(1, 'D')).astype(int)
    
    artist['last_listen'] = pd.to_datetime(artist.last_listen, errors='coerce')
    artist['lifetime'] = artist.last_listen - artist.first_listen 
    artist['lifetime'] = (artist.lifetime / np.timedelta64(1, 'D')).astype(int)
    artist['lifetime_bin'] =  pd.cut(artist.lifetime, [-1, 31,364, 365, 730, 900], labels=['1 month', '>1 month',
                                                                                        '1 year','1-2 year','>2 years'])
    
    print(artist.head)
    return artist
    

In [ ]:
def get_collinearity(data):
    X = data[['listening_days','lifetime','num_albums','num_songs','num_artists','num_playlists','user_stream_count','first_day','age']]
    corr = X.corr()
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.0, vmin=-1.0, center=0,annot=True,
               square=True, linewidths=.5, cbar_kws={"shrink": .5})

def cohort_period(df):
    df['cohort_period'] = np.arange(len(df)) + 1
    return df

def cohort_analysis(df):
    cohort = get_cohort_table(df)
    cohort = cohort.groupby(level=0).apply(cohort_period)
    ret = get_retention(cohort)
    export_retention_chart(ret,show_plot=True)
    return ret, cohort 

In [ ]:
def set_cohort_group(df):
	'''
	Identify cohort group based on date of first streaming
	'''
	df.set_index('customer_id', inplace=True)
    # customer first & last day
	df['first_listen'] = df.groupby(level=0)['date'].min()
	df['last_listen'] = df.groupby(level=0)['date'].max()
	df['cohort_group'] = df['first_listen']
	df['cohort_group'] = df['cohort_group'].astype(str)
	df.reset_index(inplace=True)
	return df

def get_cohort_table(df):
	grouped = df.groupby(['cohort_group','date'])
	cohort = grouped.agg({'customer_id': pd.Series.nunique,
							'user_stream_count': pd.Series.sum})
	# rename columns
	cohort.rename(columns = {'customer_id': 'total_customers',
							 'user_stream_count': 'total_streams'},inplace=True)
	return cohort

def get_cohort(df):
	grouped = df.groupby(['cohort_group'])
	# aggregate total athletes & activities
	# use count of athlete_id instead of block_visit_id to account for signup interaction
	cohort = grouped.agg({'athlete': pd.Series.nunique,
							'athlete_id': pd.Series.count})
	# rename columns
	cohort.rename(columns = {'athlete': 'total_users',
							 'athlete_id': 'total_activities'},inplace=True)
	return cohort


def get_retention(cohorts):
	# reindex the DataFrame 
	cohorts.reset_index(inplace=True)
	cohorts.set_index(['cohort_group', 'cohort_period'], inplace=True)
	cohort_group_size = cohorts['total_customers'].groupby(level=0).first()
	user_retention = cohorts['total_customers'].unstack(0).divide(cohort_group_size, axis=1)
	return user_retention

def export_retention_chart(retention,show_plot=False):
	sns.set(style='white', font_scale=1)
	fig = plt.figure(figsize=(30, 30))
	plt.title('Cohorts Analysis: User Retention')
	chart = sns.heatmap(retention.T, mask=retention.T.isnull(), annot=True, fmt='.0%',cmap='Blues');
	chart.set_ylabel('Cohort Group') 
	chart.set_xlabel('Cohort Period')
	#fig.savefig('output/retention.png', dpi=150)
	print('plot saved')
	if show_plot==True:
		plt.show()
	elif show_plot==False:
		pass

In [ ]:
data = read_data()

missing values

In [ ]:
msno.matrix(data.sample(10000))

## Plotting Trend

In [ ]:
album = data[(data.stream_source=='album')&(data.year>2014)].groupby('date')['stream_source'].count()
playlist = data[(data.stream_source=='others_playlist')&(data.year>2014)].groupby('date')['stream_source'].count()
all_streams = data[(data.stream_source=='others_playlist')].groupby('date')['stream_source'].count()
lines = ['-', '--']
fig, ax = plt.subplots(figsize=(16,10))
frames = {'album':album, 'playlist': playlist}
ax.set_prop_cycle('color',plt.cm.tab10(np.linspace(0,1,9)))
i=0
for k,v in frames.items():
    ax.plot(v,label=k,linestyle=lines[i])
    i+=1

legend = ax.legend(loc=2, shadow=True, fontsize=18)

# Put a nicer background color on the legend.
#legend.get_frame().set_facecolor('#00FFCC')
plt.title('Streaming from albums and playlists', fontsize=18)
#plt.xticks(np.arange(1, 20.1, 1))
plt.xticks(np.arange(1, 35, step=5))
ax.tick_params(labelsize=12)
#plt.xlim(1, 20)
plt.ylabel('stream count',fontsize=18);
plt.xlabel('date',fontsize=18)
plt.show()

In [ ]:
fig.savefig('album_playlist_stream_trend',dpi=150)

In [ ]:
data[(data.year>2014) & (data.month.isin(np.arange(7)))].groupby('year')['stream_source'].count()

In [ ]:
np.arange(7)

In [ ]:
data[(data.year>2014) & (data.stream_source=='album') & (data.month.isin(np.arange(7)))].groupby('year')['stream_source'].count(), 

In [ ]:
data[(data.year>2014) & (data.stream_source=='others_playlist') & (data.month.isin(np.arange(7)))].groupby('year')['stream_source'].count()

In [ ]:
album = data[(data.stream_source=='album')&(data.year>2014)].groupby('date')['album_name'].nunique()
playlist = data[(data.stream_source=='others_playlist')&(data.year>2014)].groupby('date')['playlist_id'].nunique()


fig, ax = plt.subplots(figsize=(16,10))
frames = {'album':album, 'playlist': playlist}
ax.set_prop_cycle('color',plt.cm.tab10(np.linspace(0,1,9)))
lines = ['-', '--']
i=0
for k,v in frames.items():
    ax.plot(v,label=k,linestyle=lines[i])
    i+=1

legend = ax.legend(loc=2, shadow=True, fontsize=18)

# Put a nicer background color on the legend.
#legend.get_frame().set_facecolor('#00FFCC')
plt.title('Total unique albums and playlists listened', fontsize=18)
#plt.xticks(np.arange(1, 20.1, 1))
plt.xticks(np.arange(0, 35, step=6))
ax.tick_params(labelsize=16)
#plt.xlim(1, 20)
plt.ylabel('total',fontsize=18);
plt.xlabel('date',fontsize=18)
plt.show()

In [ ]:
fig.savefig('album_playlist_unique_count',dpi=150)

In [ ]:
album = data[(data.stream_source=='album')&(data.year>2014)].groupby('date')['customer_id'].nunique()
playlist = data[(data.stream_source=='others_playlist')&(data.year>2014)].groupby('date')['customer_id'].nunique()


fig, ax = plt.subplots(figsize=(16,10))
frames = {'album':album, 'playlist': playlist}
ax.set_prop_cycle('color',plt.cm.tab10(np.linspace(0,1,9)))
lines = ['-', '--']
i = 0
for k,v in frames.items():
    ax.plot(v,label=k,linestyle=lines[i])
    i+=1

legend = ax.legend(loc=2, shadow=True, fontsize=18)

# Put a nicer background color on the legend.
#legend.get_frame().set_facecolor('#00FFCC')
plt.title('Total listeners', fontsize=18)
#plt.xticks(np.arange(1, 20.1, 1))
plt.xticks(np.arange(0, 35, step=6))
ax.tick_params(labelsize=16)
#plt.xlim(1, 20)
plt.ylabel('Number of customer ID',fontsize=18);
plt.xlabel('date',fontsize=18)
plt.show()

In [ ]:
fig.savefig('album_playlist_listeners_count',dpi=150)

## Do they cannibalize each other?

In [ ]:
data.stream_source.value_counts()

In [ ]:
# count of play of tracks from album & playlist
album = data[(data.stream_source=='album')&(data.year>2014)].groupby('track_id')['stream_source'].count().to_dict()
playlist = data[(data.stream_source=='others_playlist')&(data.year>2014)].groupby('track_id')['stream_source'].count().to_dict()

data['played_in_album'] = data.track_id.map(album).fillna(0).astype('int')
data['played_in_playlist'] = data.track_id.map(playlist).fillna(0).astype('int')

In [ ]:
# total track stream
data = get_songs_stream_count(data)

In [ ]:
data.head()

In [ ]:
# give number in place of track id for viz simplicity
def set_song_num(df):
    tracks = df.drop_duplicates(subset='track_id')
    tracks['track_num'] = np.arange(len(tracks)) + 1
    df = df.merge(tracks[['track_id','track_num']],on='track_id',how='left')
    return df

In [ ]:
data = set_song_num(data)
data.head()

transform data for scatterplot viz

In [ ]:
data.track_id.value_counts()

In [ ]:
track_album = data[data.stream_source=='album']
track_album['play_count'] = track_album.track_id.map(track_album.track_id.value_counts())
track_album = track_album.drop_duplicates(subset=['track_id','stream_source'])

track_playlist = data[data.stream_source=='others_playlist']
track_playlist['play_count'] = track_playlist.track_id.map(track_playlist.track_id.value_counts())
track_playlist= track_playlist.drop_duplicates(subset=['track_id','stream_source'])


In [ ]:
tracks = pd.concat([track_playlist,track_album],ignore_index=True)

In [ ]:
tracks = data[data.stream_source.isin(['album','others_playlist'])]

num_sources = tracks.groupby('track_id')['stream_source'].nunique().to_dict()
tracks['num_sources'] = tracks.track_id.map(num_sources)
# only consider track that has been played in both stream sources
tracks = tracks[tracks.num_sources==2]

In [ ]:
tracks.num_sources.unique()

In [ ]:

sns.lmplot( x="played_in_playlist", y="played_in_album", data=data[data.year>2014].drop_duplicates(subset=['track_id']), fit_reg=True,size=10)
#ax.set_ylabel('total streaming from album')
#ax.set_xlabel('total streaming from playlist')
# Move the legend to an empty part of the plot
plt.xlabel('total streaming from playlist',fontsize=14)
plt.ylabel('total streaming from album',fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


In [ ]:
# Use the 'hue' argument to provide a factor variable
sns.lmplot( x="played_in_playlist", y="played_in_album", data=data[(data.year>2014) & (data.played_in_playlist<=2500)].drop_duplicates(subset=['track_id']), fit_reg=True,size=10)
#ax.set_ylabel('total streaming from album')
#ax.set_xlabel('total streaming from playlist')
# Move the legend to an empty part of the plot
plt.xlabel('total streaming from playlist',fontsize=14)
plt.ylabel('total streaming from album',fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


In [ ]:
tracks[(tracks.played_in_playlist<=2500) & (tracks.played_in_album<100)]

In [ ]:
corr = data[data.year>2014].drop_duplicates(subset=['track_id'])[['played_in_playlist','played_in_album','song_stream_count']].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8, 8))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap="Blues", vmax=1.0, vmin=-1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True)
#sns.set(font_scale=1.3)
sns.set_style('white')

In [ ]:
data.customer_id.nunique()

In [ ]:
data[data.year==2015].groupby('month')['customer_id'].nunique()

In [ ]:
data.describe()

In [ ]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
data['age'] = data.year - data.birth_year

In [ ]:
data['age'] = data.age.apply(convert_to_int)

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(data.drop_duplicates(subset=['customer_id'],keep='last')) 

In [ ]:
data.album_name.nunique(), data.artist_name.nunique(), data.track_id.nunique(), data.customer_id.nunique()

In [ ]:
data.playlist_id.nunique()

In [ ]:
len(data)

In [ ]:
data['day'] = data.date.dt.dayofyear
data['week'] = data.date.dt.weekofyear

In [ ]:
data.date.min(), data.date.max()

In [ ]:
data.drop_duplicates(subset='customer_id').gender.value_counts()

In [ ]:
cust

In [ ]:
cust = data.drop_duplicates(subset='customer_id')

In [ ]:
cust[cust.access=='basic-desktop'][['access','year','stream_device','stream_os','mobile']].sort_values(by='year')

In [ ]:
n_access = data.groupby('customer_id')['access'].nunique().to_dict()
data['n_access'] = data.customer_id.map(n_access)
data.n_access.unique()

In [ ]:
data[data.n_access>1].drop_duplicates(subset='customer_id').access.value_counts()

In [ ]:
data[data.access=='basic-desktop'].drop_duplicates(subset='customer_id').n_access.value_counts()

In [ ]:
cust = data[data.n_access>1].drop_duplicates(subset=['customer_id','access'])
cust.head()

In [ ]:
basic_users = cust[cust.access=='basic-desktop'].customer_id.unique()
cust = cust[cust.customer_id.isin(basic_users)]
cust

In [ ]:
cust[['customer_id','access']].sort_values(by='customer_id')

In [ ]:
cust[cust.access=='free'][['access','year','stream_device','stream_os','mobile']]

In [ ]:
cust = data.drop_duplicates(subset='customer_id')
#pandas_profiling.ProfileReport(cust) 

In [ ]:
data[data.customer_id.isin(cust)].gender.value_counts()

In [ ]:
data[data.customer_id.isin(cust)].gender.value_counts()

In [ ]:
msno.matrix(df.drop_duplicates(subset='customer_id').sample(5000))

In [ ]:
data.columns

In [ ]:
cust = data[data.year==2015].customer_id.drop_duplicates()
cust

In [ ]:
data[]

In [ ]:
cust.to_csv('customer2015_dataset2.csv',index=False)

## sample dataset 2

In [ ]:
df = pd.read_pickle('/project/samples/new/sample_complete_105k.pickle')

In [ ]:
df = df.sort_values(by='logtime')

In [ ]:
import gc
gc.collect()

In [ ]:
basics = df[df.access=='basic-desktop'].customer_id.unique()
df = df[~df.customer_id.isin(basics)]
df.customer_id.nunique()


In [ ]:
df = df.dropna(subset=['gender'])
df.customer_id.nunique()


In [ ]:
cust = df['customer_id']
cust.to_csv('/project/customers_df/final/dataset2_customers.csv',index=False)

In [ ]:
df = pd.read_pickle('/project/samples/new/sample_mixed_105k.pickle')
df.customer_id.nunique()

In [ ]:
def get_num_access(df):
    num_access = df.groupby('customer_id')['access'].nunique().to_dict()
    df['num_access'] = df['customer_id'].map(num_access)
    return df

In [ ]:
df = get_num_access(df)
df = df[df.num_access==1]

In [ ]:
df.customer_id.nunique()

In [ ]:
df2 = pd.read_pickle('/project/samples/new/sample_mixed_90k_cleaned.pickle')

In [ ]:
df = pd.read_pickle('/project/samples/new/sample_nonlisteners_45k.pickle')
df = get_num_access(df)

In [ ]:
df = df[df.num_access==1]
df.customer_id.nunique()

In [ ]:
import gc
gc.collect()

In [ ]:
new_c = df[~df.customer_id.isin(df2.customer_id.unique())].drop_duplicates(subset=['customer_id'])
len(new_c)

In [ ]:
from sklearn.utils import resample

In [ ]:
new_c = resample(new_c,n_samples=14000, replace=False)
new_c.gender.value_counts(),new_c.access.value_counts()

In [ ]:
new_c=pd.DataFrame()
new_c =None

In [ ]:
df = df[df.customer_id.isin(new_c.customer_id)]
df.customer_id.nunique()

In [ ]:
df.to_pickle('/project/samples/new/additional_sample_nonlisteners_14k.pickle')

In [ ]:
df = pd.concat([df,df2],ignore_index=True)
df.customer_id.nunique()

In [ ]:
df = df.sort_values(by='logtime')

In [ ]:
df = pd.read_pickle('/project/samples/new/sample_mixed_105k_cleaned2.pickle')

In [ ]:
df[df.track_id == 'f72fa60c8d9848a393d8ac4bbaa866ef'].customer_id.nunique()

In [ ]:
gc.collect()

In [ ]:
df['listen_to_track'] = df.where(df.track_id == 'f72fa60c8d9848a393d8ac4bbaa866ef',0)
df['listen_to_track'] = df['listen_to_track'].fillna(1)
df.head()

In [ ]:
df.head()

In [ ]:
pandas_profiling.ProfileReport(df.drop_duplicates(subset='customer_id')) 

In [ ]:
len(df)

In [ ]:
df.customer_id.nunique(),df.artist_name.nunique(),df.album_name.nunique(),df.track_id.nunique()

In [ ]:
df.playlist_id.nunique()

## boxplot - customers

In [ ]:
df = pd.read_pickle('/project/samples/new/sample_mixed_105k_cleaned2.pickle')
df.head()

In [ ]:
import seaborn as sns
sns.set(style="ticks", palette="pastel")

ax = sns.boxplot(x=df.user_stream_count)


In [ ]:
# listeners and non-listeners
fig, ax = plt.subplots(1,2,figsize=(14,5))
sns.distplot(df[df.track_id=='f72fa60c8d9848a393d8ac4bbaa866ef'] .user_stream_count, ax=ax[0])
sns.distplot(df[df.track_id!='f72fa60c8d9848a393d8ac4bbaa866ef'].user_stream_count,ax=ax[1])
ax[0].set_xlabel('stream count of "See You Again" listeners')
ax[1].set_xlabel('stream count of "See You Again" non-listeners')

In [ ]:
import gc
gc.collect()

In [ ]:
# without outliers
# listeners and non-listeners
fig, ax = plt.subplots(1,2,figsize=(14,5))
sns.distplot(df[(df.track_id=='f72fa60c8d9848a393d8ac4bbaa866ef') & (df.user_stream_count<1000)] .user_stream_count, ax=ax[0])
sns.distplot(df[(df.track_id!='f72fa60c8d9848a393d8ac4bbaa866ef') & (df.user_stream_count<1000)].user_stream_count,ax=ax[1])
ax[0].set_xlabel('stream count of "See You Again" listeners')
ax[1].set_xlabel('stream count of "See You Again" non-listeners')

In [ ]:
# albums and playlists
fig, ax = plt.subplots(1,2,figsize=(14,5))
sns.distplot(df[(df.stream_source=='others_playlist') & (df.track_id=='f72fa60c8d9848a393d8ac4bbaa866ef')].user_stream_count, ax=ax[0])
sns.distplot(df[(df.stream_source=='album') & (df.track_id=='f72fa60c8d9848a393d8ac4bbaa866ef')].user_stream_count,ax=ax[1])
ax[0].set_xlabel('stream count of playlists listeners')
ax[1].set_xlabel('stream count of albums listeners')

In [ ]:
# without outliers

# listeners and non-listeners
fig, ax = plt.subplots(1,2,figsize=(14,5))
sns.distplot(df[(df.track_id=='f72fa60c8d9848a393d8ac4bbaa866ef') & (df.user_stream_count<2000)] .user_stream_count, ax=ax[0])
sns.distplot(df[(df.track_id!='f72fa60c8d9848a393d8ac4bbaa866ef') & (df.user_stream_count<2000)].user_stream_count,ax=ax[1])
ax[0].set_xlabel('stream count of "See You Again" listeners')
ax[1].set_xlabel('stream count of "See You Again" non-listeners')

In [ ]:
gc.collect()

## boxplot - stream source

In [ ]:
df.columns

In [ ]:
def get_stream_source_means(df):
    total_play = df.groupby('stream_source')['stream_source'].count()
    total_customers = df.groupby('stream_source')['customer_id'].nunique()
    

In [ ]:
gc.collect()

In [ ]:
grouped = df.groupby('stream_source')['index'].count().rename(columns={'index':'play_count'},inplace=True)
total_customers = df.groupby('stream_source')['customer_id'].nunique()


In [ ]:
grouped = grouped.reset_index()
total_customers = df.groupby('stream_source')['customer_id'].nunique().to_dict()
grouped['customers'] = grouped.stream_source.map(total_customers)

In [ ]:
grouped['dataset_flag'] = 'dataset_2'
grouped.rename(columns={0:'play_count'},inplace=True)
grouped

In [ ]:
grouped

In [ ]:
df = read_data()

In [ ]:
grouped2 = df.groupby('stream_source')['customer_id'].count().rename(columns={'customer_id':'play_count'},inplace=True)
total_customers = df.groupby('stream_source')['customer_id'].nunique().to_dict()
grouped2 = grouped2.reset_index()
grouped2['customers'] = grouped2.stream_source.map(total_customers)

In [ ]:
grouped2['dataset_flag'] = 'dataset_1'
grouped2

In [ ]:
grouped2.rename(columns={0:'play_count'},inplace=True)

In [ ]:
grouped2

In [ ]:
grouped = pd.concat([grouped,grouped2],ignore_index=True)
grouped

In [ ]:
grouped['avg_play_per_customer'] = grouped.play_count.divide(grouped.customers)
grouped

In [ ]:
playlist = pd.DataFrame(df.playlist_id.value_counts()).reset_index().rename(columns={'playlist_id':'stream_count'})
album = pd.DataFrame(df.album_name.value_counts()).reset_index().rename(columns={'album_name':'stream_count'})
playlist['stream_source'] = 'playlist'
album['stream_source'] = 'album'

In [ ]:
df_box = pd.concat([playlist,album])
df_box

In [ ]:
import seaborn as sns
sns.set(style="ticks", palette="pastel")

ax = sns.boxplot(x="stream_source", y="stream_count", data=df_box)
ax.set_title('Boxplot stream count of dataset 1')

In [ ]:
gc.collect()

In [ ]:
playlist2 = pd.DataFrame(df.playlist_id.value_counts()).reset_index().rename(columns={'playlist_id':'stream_count'})
album2 = pd.DataFrame(df.album_name.value_counts()).reset_index().rename(columns={'album_name':'stream_count'})
playlist2['stream_source'] = 'playlist'
album2['stream_source'] = 'album'

In [ ]:
df_box2 = pd.concat([playlist2,album2])
df_box2

In [ ]:
sns.set(style="ticks", palette="pastel")

ax = sns.boxplot(x="stream_source", y="stream_count", data=df_box2)
ax.set_title('Boxplot stream count of dataset 2')

#### without outliers

In [ ]:
sns.boxplot(x="stream_source", y="stream_count", data=df_box2,showfliers=False)

In [ ]:
df_box2[df_box2.stream_source=='playlist'].stream_count.mean()

In [ ]:
sns.distplot(df_box2[(df_box2.stream_source=='playlist')].stream_count),sns.distplot(df_box2[(df_box2.stream_source=='album')].stream_count)

In [ ]:
sns.distplot(df_box[(df_box.stream_source=='playlist')].stream_count),sns.distplot(df_box[(df_box.stream_source=='album')].stream_count)

In [ ]:
df_b = play.merge

## track repeat count

In [ ]:
import gc
gc.collect()

In [ ]:
track_id = 'f72fa60c8d9848a393d8ac4bbaa866ef'

In [ ]:
def get_repeated_stream_count(df):
    # count of repeated track streaming
    repeated = df.groupby(['customer_id','track_id'])['logtime'].count()
    df = df.merge(repeated.reset_index(), on=['customer_id','track_id'],how='left')
    df.rename(columns = {'logtime_x':'logtime', 'logtime_y':'track_repeat_count'}, inplace = True)
    return df

In [ ]:
df = get_repeated_stream_count(df)

In [ ]:
ax = df[df.track_id!=track_id].drop_duplicates(subset=['customer_id','track_id']).groupby('first_stream_source')['track_repeat_count'].mean().plot.barh(color='darkblue',figsize=(8,6),title='Average track repeat count per first source')
val = df.drop_duplicates(subset=['customer_id','track_id']).groupby('first_stream_source')['track_repeat_count'].mean().values.tolist()
ax.set_ylabel('First Stream Source')
ax.set_xlabel('Number of repeat')
for n,row in enumerate(val):
    ax.text(row-0.2,n-0.05, round(row,2), color='white', ha="center",fontsize=12)